# KNN - sklearn.neighbors
- 각 데이터들 간의 거리를 측정하여 가까운 K개의 다른 데이터의 레이블을 참조하여 분류하는 방법
- 거리 계산 -> 유클리디안 거리 or 민코브스키 방법 
- 중요한 파라미터 -> '몇 개의 케이스들을 기준으로 동일 범주, 동일 값을 분류 혹은 예측할 것인가'
- 최적의 K수는 일반적으로 3~10 범위 내에서
- K값이 작을수록 정교한 분류와 예측이 가능하지만 과대적합을 일으킬 가능성이 커지고 K값이 클 수록 과소적합을 일으킬 가능성이 있으므로 적합한 k수를 찾기 위해 GridSearchCV

## KNN - 분류

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

df = pd.read_csv('../datasets/빅분기 실기 데이터/breast-cancer-wisconsin.csv', encoding='utf-8')

In [3]:
X = df.drop('Class', axis=1)
y = df[['Class']]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=11)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_scaled_train, y_train)
pred = model.predict(X_scaled_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))
print(model.score(X_scaled_test, y_test))

0.975609756097561
0.975609756097561


In [7]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_test = confusion_matrix(y_test, pred)
print('테스트데이터 오차행렬:\n', confusion_test)

테스트데이터 오차행렬:
 [[129   4]
 [  1  71]]


In [8]:
cfreport_test = classification_report(y_test, pred)
print('분류예측 레포트"\n', cfreport_test)

분류예측 레포트"
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       133
           1       0.95      0.99      0.97        72

    accuracy                           0.98       205
   macro avg       0.97      0.98      0.97       205
weighted avg       0.98      0.98      0.98       205



### GridSearchCV
- 하이퍼파라미터인 n_neighbors값 조정하면서 더 좋은 예측 모델 탐색

In [9]:
param_grid = {'n_neighbors':[1,3,5,7,9,11]}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [12]:
print('Best Parameter: {}'.format(grid_search.best_params_))
print('Best Score: {:.4f}'.format(grid_search.best_score_))
print('TestSet Score: {:.4f}'.format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 5}
Best Score: 0.9749
TestSet Score: 0.9756


### KNN 정리
- KNN은 모델기반 알고리즘이 아닌 케이스기반 알고리즘
- 즉, 입력되는 데이터에 따라 결과와 성능이 크게 달라짐
- 새로운 데이터를 학습하는 시간이 오래 걸려 실시간 머신러닝이 필요한 분야에는 적합하지 않을 수 있음

## KNN - 회귀

In [14]:
df = pd.read_csv('../datasets/빅분기 실기 데이터/house_price.csv')

In [16]:
X = df.drop('house_value', axis=1)
y = df[['house_value']]

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

In [23]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(x_scaled_train, y_train)
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.5859974733823394

### RMSE 계산

In [24]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_test = mean_squared_error(y_test, pred_test)
print('테스트 데이터 RMSE:', np.sqrt(MSE_test))

테스트 데이터 RMSE: 61796.71762314058


### GridSearchCV

In [25]:
param_grid = {'n_neighbors' : [1,3,5,7,9,11]}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search.fit(x_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [26]:
print('Best Parameter: {}'. format(grid_search.best_params_))
print('Best Score: {:.4f}'.format(grid_search.best_score_))
print('Testset Score: {:.4f}'.format(grid_search.score(x_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 11}
Best Score: 0.6092
Testset Score: 0.6182


### KNN 정리
- 이 경우에는 K 값을 더 늘려서 가까운 주변의 여러 데이터의 평균으로 주택 가격을 예측하는 것이 모델의 성능 및 일반화에 모두 좋은 것으로 파악됨